# Exercises - API consumption

## Useful links

- The [Star Wars API](https://swapi.dev/) (swapi.dev)
- The [Request module](https://docs.python-requests.org/en/master/) (docs.python-requests.org)

## Setup

1. You need to install the `requests` module and the `psycopg2` module in your python virtualenv:
   1. `python -m pip install requests`
   1. `python -m pip install psycopg2` If `psycopg2` doesn't work, then try to install `psycopg2-binary`.
2. You need to have access to a postgres server. Either the one you have installed on your Windows machine, or a server running inside a docker container.

Please note for this exercise that the SWAPI is limited to 10,000 requests for a given day for a given IP address. Make sure to not exeed that number.


In [19]:
import psycopg2
import requests
import json
import os
from flask import Flask



## Exercise 1 - Familiarize yourself with the API and the docs

1. Familiarise yourself with the SWAPI using the tool in the front page. Look at the responses, build a sense of how the data is structured and how it can be scraped.
2. Skim throught the documentation page [link here](https://swapi.dev/documentation).
3. Write a Python function that retrieves a single resource from the SWAPI given its resource URL, and returns the response object.


In [13]:
URL = 'https://swapi.dev/api/'

def get_resource(resource):
    respons = requests.get(f'{URL}{resource}')
    return respons
# print(get_resource('people/1').json())

{'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'skin_color': 'fair', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'homeworld': 'https://swapi.dev/api/planets/1/', 'films': ['https://swapi.dev/api/films/1/', 'https://swapi.dev/api/films/2/', 'https://swapi.dev/api/films/3/', 'https://swapi.dev/api/films/6/'], 'species': [], 'vehicles': ['https://swapi.dev/api/vehicles/14/', 'https://swapi.dev/api/vehicles/30/'], 'starships': ['https://swapi.dev/api/starships/12/', 'https://swapi.dev/api/starships/22/'], 'created': '2014-12-09T13:50:51.644000Z', 'edited': '2014-12-20T21:17:56.891000Z', 'url': 'https://swapi.dev/api/people/1/'}



## Exercise 2 - Parse the schemas

1. Try to use the function above to run `/schema/` for all resource types in the SWAPI (f.ex. `/people/schema/`). Check out a description of schemas [here](https://swapi.dev/documentation#schema). If this doesn't work, you need to parse the schema by hand. For each resourse type (e.g. people), use a resourse you think has a complete number of attributes.


In [14]:
# get_resource('people').text

'{"count":82,"next":"https://swapi.dev/api/people/?page=2","previous":null,"results":[{"name":"Luke Skywalker","height":"172","mass":"77","hair_color":"blond","skin_color":"fair","eye_color":"blue","birth_year":"19BBY","gender":"male","homeworld":"https://swapi.dev/api/planets/1/","films":["https://swapi.dev/api/films/1/","https://swapi.dev/api/films/2/","https://swapi.dev/api/films/3/","https://swapi.dev/api/films/6/"],"species":[],"vehicles":["https://swapi.dev/api/vehicles/14/","https://swapi.dev/api/vehicles/30/"],"starships":["https://swapi.dev/api/starships/12/","https://swapi.dev/api/starships/22/"],"created":"2014-12-09T13:50:51.644000Z","edited":"2014-12-20T21:17:56.891000Z","url":"https://swapi.dev/api/people/1/"},{"name":"C-3PO","height":"167","mass":"75","hair_color":"n/a","skin_color":"gold","eye_color":"yellow","birth_year":"112BBY","gender":"n/a","homeworld":"https://swapi.dev/api/planets/1/","films":["https://swapi.dev/api/films/1/","https://swapi.dev/api/films/2/","htt


## Exercise 3 - Create tables per resource type

1. Create a new database for these exercises.
1. For each resource type, create corresponding tables in your new database by using the schema you have started parsing. If you have to do the schema parsing manually, you will probably need to iterate multiple times here.


In [15]:
conn = psycopg2.connect("dbname=postgres user=postgres password=mysecretpassword", port=5544)
cur = conn.cursor()

path = os.listdir('C://Users//magnu//Documents//Codeprojects//Code//AW//ETL//schema')
titles = []
for i in path:
    titles.append(i.split('.')[0])


def create_table(title, dic):
    st = ""
    for name,datatype in dic.items():
        st += f', {name} {datatype}'
    cur.execute(
        f"CREATE TABLE {title} (id serial PRIMARY KEY{st});")
    conn.commit()

columns_dic = {}

def get_schemas(path, columns_dic):
    for i in path:
        with open(f'C://Users//magnu//Documents//Codeprojects//Code//AW//ETL//schema//{i}', 'r') as file:
            js = json.load(file)
        newdic = {}
        for key, value in js['properties'].items():
            if value['type'] == 'string':
                newdic[key] = 'varchar'
            if value['type'] == 'integer':
                newdic[key] = 'integer'
            else:
                newdic[key] = 'varchar'
        title = i.split('.')[0]
        columns_dic[title] = newdic
        create_table(title, newdic)

# get_schemas(path, columns_dic)


## Exercise 3 - Function to populate

5. For each resource type (e.g. planets), write a Python function to populate the corresponding table based on a response object.
6. Try it out! Can you populate at least one resource in each table?


In [16]:
def write_resource(title, dic):
    keys = ''
    values = ''
    for key, value in dic.items():
        stre = ''
        if value == []:
            value = ''
        if not isinstance(value, (str, int)):
            if value:
                for element in value:
                    if element:
                        stre += element
                        value = stre
        if isinstance(value, str):
            strenge = ''
            for char in value:
                if char == "'":
                    continue
                else:
                    strenge += char
            value = strenge
        keys += f'{key},'
        values += f"""'{value}',"""
    print(f'''insert into {title}({keys[:-1]}) values({values[:-1]})''')
    cur.execute(f'insert into {title}({keys[:-1]}) values({values[:-1]})')
    conn.commit()




        



## Bonuse exercise - Scraping the entire API

1. Write a Python program that scrapes the entire API, starting with only the information in the Luke Skywalker resource. Ensure that your program never requests the same resource more than once. Discuss with a classmate how you should build your program before you start writing any code.
2. Find some interesting statistics about the Star Wars universe based on your brand new database!


In [26]:
def fill_dict():
    ls = {}
    for i in titles:
        les = []
        for x in range(1000):
            if get_resource(f'{i}/{x}').ok:
                les.append(get_resource(f'{i}/{x}').json())
            continue
        ls[i] = les
    return ls

# data_dict = fill_dict()





## Bonus exercise - Automize creation of resource type tables

1. Consider task 3 again. Can you figure out a way to create a function that accepts the path to a resource schema, and creates the table for the resource type in the database without any further user intervention?


In [18]:
# for title in titles:
#         for dict in data_dict[title]:
#                 write_resource(title, dict)




insert into films(title,episode_id,opening_crawl,director,producer,release_date,characters,planets,starships,vehicles,species,created,edited,url) values('A New Hope','4','It is a period of civil war.
Rebel spaceships, striking
from a hidden base, have won
their first victory against
the evil Galactic Empire.

During the battle, Rebel
spies managed to steal secret
plans to the Empires
ultimate weapon, the DEATH
STAR, an armored space
station with enough power
to destroy an entire planet.

Pursued by the Empires
sinister agents, Princess
Leia races home aboard her
starship, custodian of the
stolen plans that can save her
people and restore
freedom to the galaxy....','George Lucas','Gary Kurtz, Rick McCallum','1977-05-25','https://swapi.dev/api/people/1/https://swapi.dev/api/people/2/https://swapi.dev/api/people/3/https://swapi.dev/api/people/4/https://swapi.dev/api/people/5/https://swapi.dev/api/people/6/https://swapi.dev/api/people/7/https://swapi.dev/api/people/8/https://swapi.dev/api/

In [25]:



app = Flask('__name__')

@app.route('/')
def root_page():
    return 'Welcome to root'

@app.route('/films')
def films():
    pass

cur.execute("select * from films;")
select = cur.fetchall()
for row in select:
    print("code: {}".format(row[0]))
    print("name: {}".format(row[1]))
    print("concatenated_column: {}".format(row[2]))

AttributeError: module 'psycopg2' has no attribute 'extras'